### Knock61: Directory 生成をして機械学習用 Data を読み込もう

In [1]:
# Directory 作成
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [2]:
# 機械学習用 Data の読み込み
import pandas as pd
ml_data_file = 'ml_base_data.csv'
ml_data = pd.read_csv(os.path.join(input_dir, ml_data_file))
ml_data.head(3)

,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0,0.0,1147,945,202,841,306,844,303,...,101,95,107,106,100,108,109,96,34.110053,201904
1,さいたま南店,1.0,1.0,1504,1217,287,1105,399,1104,400,...,143,142,137,130,113,140,132,155,35.337716,201904
2,さいたま緑店,1.0,1.0,1028,847,181,756,272,756,272,...,95,102,82,90,93,95,95,84,34.291617,201904


### Knock62: Categorical 変数の対応をしよう
- One-hot-encoding: Categorical 変数を、特定の Category に属していたら 1 の Flag を立てる形式

In [3]:
from IPython.display import display

# One-hot-encoding
category_data = pd.get_dummies(ml_data['store_name'], prefix='store', prefix_sep='_')
display(category_data.head(3))

,store_あきる野店,store_さいたま南店,store_さいたま緑店,store_さいたま西店,store_つくば店,store_三浦店,store_三鷹店,store_上尾店,store_上野店,store_世田谷店,...,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店,store_麻生店
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- pandas の `get_dummies()` を使用すると簡単に One-hot-encoding ができる。
- **多重共線性の防止**: Categorical 変数は１列消すことが一般的。（すべての Flag が 0 だった場合に削除した変数と情報が特定できる為）
- 元の Data に結合する際に、One-hot-encoding の元となった変数は削除する。

In [4]:
# Categorical 変数の結合
del category_data['store_麻生店']
del ml_data['year_month']
del ml_data['store_name']
ml_data = pd.concat([ml_data, category_data], axis=1)
ml_data.columns

Index(['y_weekday', 'y_weekend', 'order', 'order_fin', 'order_cancel',
       'order_delivery', 'order_takeout', 'order_weekday', 'order_weekend',
       'order_time_11',
       ...
       'store_駒沢店', 'store_駒込店', 'store_高円寺店', 'store_高島平店', 'store_高崎店',
       'store_高座店', 'store_高津店', 'store_高田馬場店', 'store_鴻巣店', 'store_鶴見店'],
      dtype='object', length=215)

One-hot-encoding を行なうと、情報としてはわかりにくく、他で使用する際に使いにくくなる為、機械学習に投入する直前で対応すると良い。

### Knock63: 学習 Data と Test data を分割しよう
- 機械学習の目的は、未知な Data に対応せること。
- 未知な Data に対応できる Model が汎用的な良い Model とされ、**汎用性が高い** と表現される。
- 全 Data を Model 構築に使用してしまうと、その Model は未知な Data に対応できるか評価できなくなってしまうため、
    - 学習 Data
    - Test data
    に **分割** する。

#### 学習 Data と Test data の分割
学習 Data と Test data の分割比率に正解はなく、試行錯誤する要素の１つだが、
- 7:3
- 75:25
- 8:2

あたりがよく使用される。

Sample 数が少ないと分割比率による精度の差が顕著にでてくる。その場合、**交差検証** などの分割手法で正しく精度を検証することが重要になってくる。

In [6]:
# 学習 Data と Test data の分割
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(ml_data, test_size=0.3, random_state=0)
print(f"Train: {len(train_data)}件 / Test: {len(test_data)}")
print(f"Weekday Train0: {len(train_data.loc[train_data['y_weekday'] == 0])}件")
print(f"Weekday Train1: {len(train_data.loc[train_data['y_weekday'] == 1])}件")
print(f"Weekday Test0: {len(test_data.loc[test_data['y_weekday'] == 0])}件")
print(f"Weekday Test1: {len(test_data.loc[test_data['y_weekday'] == 1])}件")

print(f"Weekend Train0: {len(train_data.loc[train_data['y_weekend'] == 0])}件")
print(f"Weekend Train1: {len(train_data.loc[train_data['y_weekend'] == 1])}件")
print(f"Weekend Test0: {len(test_data.loc[test_data['y_weekend'] == 0])}件")
print(f"Weekend Test1: {len(test_data.loc[test_data['y_weekend'] == 1])}件")

Train: 1501件 / Test: 644
Weekday Train0: 685件
Weekday Train1: 816件
Weekday Test0: 290件
Weekday Test1: 354件
Weekend Train0: 708件
Weekend Train1: 793件
Weekend Test0: 295件
Weekend Test1: 349件


### Knock64: １つの Model を構築しよう

In [7]:
# 説明変数, 目的変数の作成
X_cols = list(train_data.columns)
X_cols.remove('y_weekday')
X_cols.remove('y_weekend')
target_y = 'y_weekday'
y_train = train_data[target_y]
X_train = train_data[X_cols]
y_test = test_data[target_y]
X_test = test_data[X_cols]
display(y_train.head(3))
display(X_train.head(3))

1137    1.0
971     0.0
1983    1.0
Name: y_weekday, dtype: float64

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,...,store_駒沢店,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店
1137,977,809,168,724,253,685,292,102,88,84,...,0,0,0,0,0,0,0,0,0,0
971,1099,904,195,816,283,779,320,99,102,101,...,0,0,0,0,0,0,0,1,0,0
1983,966,794,172,724,242,671,295,80,95,87,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 決定木 Model の構築
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier(random_state=0)
model1.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

- Model を定義して、`fit()` するだけで Model が構築される。
- Model を定義する際に、**乱数種** の固定を忘れない。